# Merging geostrophic velocities with direct model output
Negligable Coriolis parameter close to the equator interferes with geostrophic velocity calculations. The region 3S to 3N is replaced with interpolated monthly velocity data from direct model output.

In [ ]:
import xarray as xr
import pyfesom2 as pf
import numpy as np
from cdo import *
cdo=Cdo()
import datetime as dt
from joblib import Parallel, delayed
import os

In [ ]:
meshpath='/path/to/mesh/data/'

Concatenate the data into the full timeseries. This is necessary in order to accurately interpolate the monthly data, but requires a large amoount of memory.

In [ ]:
years=np.arange(1850,2101,1)
runs=[1,2,3,4,5]

In [ ]:
for run in runs:
    vfiles=[]
    ufiles=[]
    for year in years:
        vfiles.append('c6_ssh-vo_daily_node_r'+str(run)+'_'+str(years[year])+'.nc')
        ufiles.append('c6_ssh-uo_daily_node_r'+str(run)+'_'+str(years[year])+'.nc')
    vout='c6_ssh-vo_daily_node_r'+str(run)+'_1850-2100.nc'
    uout='c6_ssh-uo_daily_node_r'+str(run)+'_1850-2100.nc'
    cdo.copy(input=vfiles,output=vout)
    cdo.copy(input=ufiles,output=uout)

Remove February 29th and coarsen the data to 5-day means

In [ ]:
!cdo timselmean,5,0,0 -del29feb c6_ssh-uo_daily_node_r1_1850-2100.nc c6_ssh-uo_5day_node_r1_1850-2100_nol.nc
!cdo timselmean,5,0,0 -del29feb c6_ssh-uo_daily_node_r2_1850-2100.nc c6_ssh-uo_5day_node_r2_1850-2100_nol.nc
!cdo timselmean,5,0,0 -del29feb c6_ssh-uo_daily_node_r3_1850-2100.nc c6_ssh-uo_5day_node_r3_1850-2100_nol.nc
!cdo timselmean,5,0,0 -del29feb c6_ssh-uo_daily_node_r4_1850-2100.nc c6_ssh-uo_5day_node_r4_1850-2100_nol.nc
!cdo timselmean,5,0,0 -del29feb c6_ssh-uo_daily_node_r5_1850-2100.nc c6_ssh-uo_5day_node_r5_1850-2100_nol.nc

In [ ]:
!cdo timselmean,5,0,0 -del29feb c6_ssh-vo_daily_node_r1_1850-2100.nc c6_ssh-vo_5day_node_r1_1850-2100_nol.nc
!cdo timselmean,5,0,0 -del29feb c6_ssh-vo_daily_node_r2_1850-2100.nc c6_ssh-vo_5day_node_r2_1850-2100_nol.nc
!cdo timselmean,5,0,0 -del29feb c6_ssh-vo_daily_node_r3_1850-2100.nc c6_ssh-vo_5day_node_r3_1850-2100_nol.nc
!cdo timselmean,5,0,0 -del29feb c6_ssh-vo_daily_node_r4_1850-2100.nc c6_ssh-vo_5day_node_r4_1850-2100_nol.nc
!cdo timselmean,5,0,0 -del29feb c6_ssh-vo_daily_node_r5_1850-2100.nc c6_ssh-vo_5day_node_r5_1850-2100_nol.nc

Only monthly mean velocity data is available. Interpolate the monthly data temporally.

In [ ]:
#load mesh information
mesh=pf.load_mesh(meshpath)

In [ ]:
def sel_region(lolat,hilat,lolon,hilon,inclusive=True):
    inds=np.arange(len(mesh.x2))
    if inclusive==True:
        return[a for a,x,y in zip(inds,mesh.y2,mesh.x2) if x>=lolat and x<=hilat and y>=lolon and y<=hilon]
    else:
        return[a for a,x,y in zip(inds,mesh.y2,mesh.x2) if x>lolat and x<hilat and y>lolon and y<hilon]

In [ ]:
eq=sel_region(-3,3,-180,180)

In [ ]:
def interpu(i):
    int1=mond.uo[:,i].interp_like(dayd.uo[:,0],kwargs={"fill_value": "extrapolate"})
    return int1.values

def interpv(i):
    int1=mond.vo[:,i].interp_like(dayd.uo[:,0],kwargs={"fill_value": "extrapolate"})
    return int1.values

In [ ]:
#sample of the 5-day mean time series
time_data=xr.open_dataset('c6_ssh-uo_5day_node_r1_1850-2100_nol.nc')
dtindex=dayd.indexes['time'].to_datetimeindex()
dayd['time']=dtindex

In [ ]:
for run in [1,2,3,4,5]:
    intvals=[]

    #monthly data in a single dataset for interpolation
    monvfiles=[]
    #historical
    monvfiles.append('vo_Omon_AWI-CM-1-1-MR_historical_r'+str(run)+'i1p1f1_gn_185001-185012.nc')
    for year in np.arange(1851,2001,10):
        monvfiles.append('vo_Omon_AWI-CM-1-1-MR_historical_r'+str(run)+'i1p1f1_gn_'+str(year)+'01-'+str(year+9)+'12.nc')
    monvfiles.append('vo_Omon_AWI-CM-1-1-MR_historical_r'+str(run)+'i1p1f1_gn_201101-201412.nc')
    #scenario
    monvfiles.append('vo_Omon_AWI-CM-1-1-MR_ssp370_r'+str(run)+'i1p1f1_gn_201501-202012.nc')
    for year in np.arange(2021,2091,10):
        monvfiles.append('vo_Omon_AWI-CM-1-1-MR_ssp370_r'+str(run)+'i1p1f1_gn_'+str(year)+'01-'+str(year+9)+'12.nc')
    
    mond=xr.open_mfdataset(monvfiles)
    mond=mond.sel({'depth':0},drop=True) 
    
    intvals=Parallel(n_jobs=-1,batch_size=150,verbose=10) (delayed(interpv)(i) for i in eq)
    mond.close()
    dayd.close()
    
    temp=xr.DataArray(data=np.asarray(intvals),coords={'time':dtindex},
                      dims={'time','ncells'},
                      attrs={'description':'5-day mean vo data between 3S and 3N interpolated from monthly vo data',
                            'units':'m/s',
                            })
    temp=temp.to_dataset(name='vo')
    
    temp.to_netcdf('c6_vo_mon_interp_extrap_eq_1850-2100_r'+str(run)+'.nc')
    temp.close()

In [ ]:
for run in [1,2,3,4,5]:
    intvals=[]

    #monthly data in a single dataset for interpolation
    monufiles=[]
    #historical
    monufiles.append('uo_Omon_AWI-CM-1-1-MR_historical_r'+str(run)+'i1p1f1_gn_185001-185012.nc')
    for year in np.arange(1851,2001,10):
        monufiles.append('uo_Omon_AWI-CM-1-1-MR_historical_r'+str(run)+'i1p1f1_gn_'+str(year)+'01-'+str(year+9)+'12.nc')
    monufiles.append('uo_Omon_AWI-CM-1-1-MR_historical_r'+str(run)+'i1p1f1_gn_201101-201412.nc')
    #scenario
    monufiles.append('uo_Omon_AWI-CM-1-1-MR_ssp370_r'+str(run)+'i1p1f1_gn_201501-202012.nc')
    for year in np.arange(2021,2091,10):
        monufiles.append('uo_Omon_AWI-CM-1-1-MR_ssp370_r'+str(run)+'i1p1f1_gn_'+str(year)+'01-'+str(year+9)+'12.nc')
    
    mond=xr.open_mfdataset(monufiles)
    mond=mond.sel({'depth':0},drop=True) 
    
    intvals=Parallel(n_jobs=-1,batch_size=150,verbose=10) (delayed(interpu)(i) for i in eq)
    mond.close()
    dayd.close()
    
    temp=xr.DataArray(data=np.asarray(intvals),coords={'time':dtindex},
                      dims={'time','ncells'},
                      attrs={'description':'5-day mean uo data between 3S and 3N interpolated from monthly vo data',
                            'units':'m/s',
                            })
    temp=temp.to_dataset(name='uo')
    
    temp.to_netcdf('c6_uo_mon_interp_extrap_eq_1850-2100_r'+str(run)+'.nc')
    temp.close()

Replace the equator (3S to 3N) in the geostrophic dataset with the interpolated monthly values

In [ ]:
for run in [1,2,3,4,5]:
    mond=xr.open_dataset('c6_vo_mon_interp_extrap_eq_1850-2100_r'+str(run)+'.nc')
    dayd=xr.open_dataset('c6_ssh-vo_5day_node_r'+str(run)+'_1850-2100_nol.nc')
    
    data=dayd.vo.values
    data[:,eq]=mond.vo[:,:].values
    
    da=xr.DataArray(data=data,
                    dims={'time','ncells'},
                    name='vo',
                    coords={'time':dtindex},
                    attrs={'description':'5-day geostrophic vo data with 3S to 3N replaced with interpolated monthly vo',
                           'units':'m/s',
                            })
    
    ds=da.to_dataset(name='vo')
    ds=ds.assign({'time_bnds':dayd.time_bnds})
    ds=ds.assign_attrs({'description':'5day mean vo reconstructed from geostrophy and monthly data from AWICM1 CMIP6 contribution',
                'simulation':'Years 1850-2100 from historic and ssp370 simulations. Ensemble member r'+str(run)})
    
    ds.to_netcdf('c6_ssh-vo_mon_lin_eq_5day_node_r'+str(run)+'_1850-2100_nol.nc')

In [ ]:
for run in [1,2,3,4,5]:
    mond=xr.open_dataset('c6_uo_mon_interp_extrap_eq_1850-2100_r'+str(run)+'.nc')
    dayd=xr.open_dataset('c6_ssh-uo_5day_node_r'+str(run)+'_1850-2100_nol.nc')
    
    data=dayd.uo.values
    data[:,eq]=mond.uo[:,:].values
    
    da=xr.DataArray(data=data,
                    dims={'time','ncells'},
                    name='uo',
                    coords={'time':dtindex},
                    attrs={'description':'5-day geostrophic uo data with 3S to 3N replaced with interpolated monthly uo',
                           'units':'m/s',
                            })
    
    ds=da.to_dataset(name='uo')
    ds=ds.assign({'time_bnds':dayd.time_bnds})
    ds=ds.assign_attrs({'description':'5day mean uo reconstructed from geostrophy and monthly data from AWICM1 CMIP6 contribution',
                'simulation':'Years 1850-2100 from historic and ssp370 simulations. Ensemble member r'+str(run)})
    
    ds.to_netcdf('c6_ssh-uo_mon_lin_eq_5day_node_r'+str(run)+'_1850-2100_nol.nc')